## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tura,IN,25.5198,90.2201,75.18,15,0,1.81,clear sky
1,1,Pochutla,MX,15.7432,-96.4661,87.80,42,1,9.22,clear sky
2,2,Bluff,NZ,-46.6000,168.3333,62.01,83,100,4.00,light rain
3,3,Kapaa,US,22.0752,-159.3190,78.80,69,40,12.66,scattered clouds
4,4,Nantucket,US,41.2835,-70.0995,41.00,86,100,14.83,overcast clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip?  65
What is the maximum temperature you would like for your trip?  85


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1840,1840,Wote,KE,-1.7808,37.6288,66.72,73,100,0.81,overcast clouds
1841,1841,Planeta Rica,CO,8.4147,-75.5883,80.94,74,88,7.54,overcast clouds
1842,1842,Itapetinga,BR,-15.2489,-40.2478,68.16,84,2,3.60,clear sky
1843,1843,Kibiti,TZ,-7.7218,38.9375,73.33,80,100,0.38,overcast clouds
1844,1844,Hirekerur,IN,14.4667,75.3833,72.93,85,79,5.28,broken clouds


In [23]:
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tura,IN,25.5198,90.2201,75.18,15,0,1.81,clear sky
3,3,Kapaa,US,22.0752,-159.3190,78.80,69,40,12.66,scattered clouds
7,7,Rio Grande,BR,-32.0350,-52.0986,71.01,96,100,5.91,light rain
8,8,Kaputa,ZM,-8.4689,29.6619,66.88,88,99,2.06,overcast clouds
9,9,Shiyan,CN,32.6475,110.7781,70.00,20,0,3.89,clear sky


In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                829
City                   829
Country                825
Lat                    829
Lng                    829
Max Temp               829
Humidity               829
Cloudiness             829
Wind Speed             829
Current Description    829
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tura,IN,75.18,clear sky,25.5198,90.2201,
3,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,
7,Rio Grande,BR,71.01,light rain,-32.0350,-52.0986,
8,Kaputa,ZM,66.88,overcast clouds,-8.4689,29.6619,
9,Shiyan,CN,70.00,clear sky,32.6475,110.7781,
12,Bambous Virieux,MU,77.00,broken clouds,-20.3428,57.7575,
13,Rikitea,PF,79.48,clear sky,-23.1203,-134.9692,
18,Hambantota,LK,75.20,broken clouds,6.1241,81.1185,
21,Mitsamiouli,KM,82.11,overcast clouds,-11.3847,43.2844,
22,San Juan,AR,65.41,clear sky,-31.5375,-68.5364,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
#Recall that when trying to parse the data from an API request, we need to use a try-except block to handle the error with 
#a statement and continue the retrieval.

hotel_df.head(10)    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tura,IN,75.18,clear sky,25.5198,90.2201,Hotel RIKMAN Continental
3,Kapaa,US,78.80,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
7,Rio Grande,BR,71.01,light rain,-32.0350,-52.0986,Hotel Atlântico Rio Grande
8,Kaputa,ZM,66.88,overcast clouds,-8.4689,29.6619,Chelelwa Charlets
9,Shiyan,CN,70.00,clear sky,32.6475,110.7781,Fenghuang Garden Holiday Hotel
12,Bambous Virieux,MU,77.00,broken clouds,-20.3428,57.7575,Casa Tia Villa
13,Rikitea,PF,79.48,clear sky,-23.1203,-134.9692,Pension Maro'i
18,Hambantota,LK,75.20,broken clouds,6.1241,81.1185,Bungalow 63
21,Mitsamiouli,KM,82.11,overcast clouds,-11.3847,43.2844,Foyer ADM
22,San Juan,AR,65.41,clear sky,-31.5375,-68.5364,Hotel Provincial


In [36]:
# 7. Drop the rows where there is no Hotel Name.


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))